# 🚀 Deploy ADK Agent to Vertex AI Agent Engine

**Welcome to the final day of the Kaggle 5-day Agents course!**

In the previous notebook you learned how to use Agent2Agent Protocol to make your agents interoperable.

Now, let's take the final step: deploying your agents to production using [Vertex AI Agent Engine](https://docs.cloud.google.com/agent-builder/agent-engine/overview).

## 💡 Scaling Your Agent

You've built an amazing AI agent. It works perfectly on your machine. You can chat with it, it responds intelligently, and everything seems ready. But there's a problem.

> **Your agent is not publicly available!**

It only lives in your notebook and development environment. When you stop your notebook session, it stops working. Your teammates can't access it. Your users can't interact with it. And this is precisely why we need to deploy the agents!

## 🎯 What You'll Learn

In this notebook, you'll:

- ✅ Build a production-ready ADK agent
- ✅ Deploy your agent to [**Vertex AI Agent Engine**](https://docs.cloud.google.com/agent-builder/agent-engine/overview) using the ADK CLI
- ✅ Test your deployed agent with Python SDK
- ✅ Monitor and manage deployed agents in Google Cloud Console
- ✅ Understand how to add Memory to your Agent using Vertex AI Memory Bank
- ✅ Understand cost management and cleanup best practices

## ⚙️ Section 1: Setup


### 1.1: ⚠️ **Important: Prerequisites**

This notebook requires a **Google Cloud account** to deploy agents to Vertex AI Agent Engine.

**If you don't have a GCP account yet:**

 ✅ Step 1. **Create a free Google Cloud account** - [Sign up here](https://cloud.google.com/free)
- New users get **$300 in free credits** valid for 90 days on Google Cloud
- No charges during the free trial period

 ✅ Step 2. **Enable billing on your account** - Required even for free trial
- A credit card is needed for verification
- You won't be charged unless you explicitly upgrade
- This demo stays within the free tier of Agent Engine if you clean up resources promptly


 ✅ Step 3. **Understand the free trial** - Know what's included
- Check [free trial details of Google Cloud](https://cloud.google.com/free/docs/free-cloud-features#free-trial)
- Review [common questions about the free trial for Google Cloud](https://cloud.google.com/signup-faqs?hl=en#google-cloud-free-trial-faqs)

**💡 Quick Setup Guide:** Watch this [3-minute setup video](https://youtu.be/-nUAQq_evxc) for a walkthrough

### 1.2: Import components

Now, import the specific components you'll need for this notebook. This keeps your code organized and ensures we have access to the necessary building blocks.

In [1]:
import os
import random
import time
import vertexai
from vertexai import agent_engines

print("✅ Imports completed successfully")

✅ Imports completed successfully


### 1.3: Add Cloud Credentials to Secrets

1. In the top menu bar of the notebook editor, select `Add-ons` then `Google Cloud SDK`.
2. Click on `Link Account`
3. Select your Google Cloud Account
4. Attach to the notebook
   
This cell retrieves your Google Cloud credentials from Kaggle Secrets and configures them for use. These credentials allow the notebook to authenticate with Google Cloud services like Vertex AI Agent Engine.

In [ ]:
# Set up Cloud Credentials in Kaggle
user_secrets = UserSecretsClient()
user_credential = user_secrets.get_gcloud_credential()
user_secrets.set_tensorflow_credential(user_credential)

print("✅ Cloud credentials configured")

### 1.4: Set your PROJECT_ID

**Important:** Make sure to replace `"your-project-id"` with your actual Google Cloud project ID. You can find your project ID in the [Google Cloud Console](https://console.cloud.google.com/).

In [ ]:
## Set your PROJECT_ID
PROJECT_ID = "your-project-id"  # TODO: Replace with your project ID
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

if PROJECT_ID == "your-project-id" or not PROJECT_ID:
    raise ValueError("⚠️ Please replace 'your-project-id' with your actual Google Cloud Project ID.")

print(f"✅ Project ID set to: {PROJECT_ID}")

### 1.5: Enable Google Cloud APIs

For this tutorial, you'll need to enable the following APIs in the Google Cloud Console.

- Vertex AI API
- Cloud Storage API
- Cloud Logging API
- Cloud Monitoring API
- Cloud Trace API
- Telemetry API

You can [use this link to open the Google Cloud Console](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,storage.googleapis.com,logging.googleapis.com,monitoring.googleapis.com,cloudtrace.googleapis.com,telemetry.googleapis.com) and follow the steps there to enable these APIs.

---

## 🏗️ Section 2: Create Your Agent with ADK

Before we deploy, we need a functional agent to host. We'll build a **Weather Assistant** designed to serve as our sample agent.

This agent is optimized for production testing with the following configuration:

- **Model:** Uses gemini-2.5-flash-lite for low latency and cost-efficiency.
- **Tools:** Includes a `get_weather` function to demonstrate tool execution.
- **Persona:** Responds conversationally to prove the instruction-following capabilities.

This demonstrates the foundational ADK architecture we are about to package: **Agent + Tools + Instructions**.

We'll create the following files and directory structure:

```
sample_agent/
├── agent.py                  # The logic
├── requirements.txt          # The libraries
├── .env                      # The secrets/config
└── .agent_engine_config.json # The hardware specs
```